In [1]:
# Google Drive 마운트 확인
from google.colab import drive
drive.mount('/content/drive')

# 데이터셋 폴더로 이동
import os
os.chdir('/content/drive/MyDrive/2025_zolzak/')

Mounted at /content/drive


In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.1 MB/s eta 0:00:00


In [10]:
import torch
import pickle
import numpy as np
from tqdm import tqdm
from pathlib import Path
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
from ultralytics import YOLO
from ultralytics.data.augment import LetterBox
from ultralytics.utils.instance import Instances
from PIL import Image

In [5]:
with open("id_stats.pkl", "rb") as f:
    stats = pickle.load(f)

mu, cov_inv, threshold = stats["mu"], stats["cov_inv"], stats["threshold"]

In [19]:
eval_val_img_dir = "dataset_prepared_05/eval/val/images/"
eval_val_lbl_dir = "dataset_prepared_05/eval/val/labels/"
eval_test_img_dir = "dataset_prepared_05/eval/test/images/"
eval_test_lbl_dir = "dataset_prepared_05/eval/test/labels/"

In [6]:
model = YOLO("runs/train/dp05_yolov105/weights/best.pt")
model.to("cuda")
model.eval()

# Head 직전 Neck output 선택
neck_layer = model.model.model[22]

In [20]:
def load_gt_labels(label_dir):
    """라벨 파일에서 클래스 정보 로드"""
    gt_map = {}
    for txt_file in Path(label_dir).glob("*.txt"):
        with open(txt_file, "r", encoding="utf-8") as f:
            lines = f.readlines()
            fname = txt_file.stem
            # 각 라인의 첫 번째 값(클래스 ID)을 정수로 변환
            gt_map[fname] = [int(line.split()[0]) for line in lines if line.strip()]
    return gt_map

In [16]:
def custom_collate_fn(batch):
    """Instances 객체를 처리할 수 있는 custom collate function"""
    # 각 키별로 데이터 수집
    collated = {}
    for key in batch[0].keys():
        if key == "instances":
            # Instances 객체는 리스트로 유지
            collated[key] = [item[key] for item in batch]
        elif key in ["img"]:
            # 이미지는 텐서 배치로 stack
            collated[key] = torch.stack([item[key] for item in batch])
        elif key in ["cls"]:
            # 클래스는 numpy 배열의 리스트로
            collated[key] = [item[key] for item in batch]
        else:
            # 나머지는 리스트로
            collated[key] = [item[key] for item in batch]
    return collated

In [11]:
class OODDataset(Dataset):
    def __init__(self, img_dir, label_dir, gt_labels, imgsz=960):
        """
        img_dir: 이미지 폴더
        label_dir: 라벨 폴더
        gt_labels: load_gt_labels()로 로드한 GT 딕셔너리
        imgsz: 입력 이미지 크기
        """
        self.img_dir = Path(img_dir)
        self.label_dir = Path(label_dir) if label_dir else None
        self.gt_labels = gt_labels
        self.imgsz = imgsz

        # OOD 이미지만 필터링 (-1 클래스가 있는 이미지)
        self.img_paths = []
        for p in sorted(self.img_dir.iterdir()):
            if p.suffix.lower() in ['.jpg', '.jpeg', '.png']:
                stem = p.stem
                if stem in gt_labels and -1 in gt_labels[stem]:
                    self.img_paths.append(str(p))

        print(f"Found {len(self.img_paths)} OOD images")

        self.letterbox = LetterBox(new_shape=(544, 960), auto=False)

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        stem = Path(img_path).stem
        img = Image.open(img_path).convert("RGB")
        img_array = np.array(img)

        # 라벨 로드
        if self.label_dir and (self.label_dir / f"{stem}.txt").exists():
            with open(self.label_dir / f"{stem}.txt") as f:
                lines = [list(map(float, line.strip().split()))
                        for line in f if line.strip()]
            if lines:
                cls, bboxes = zip(*[(int(l[0]), l[1:]) for l in lines])
                bboxes = np.array(bboxes, dtype=np.float32)
                cls = np.array(cls)
            else:
                bboxes = np.zeros((0, 4), dtype=np.float32)
                cls = np.array([])
        else:
            bboxes = np.zeros((0, 4), dtype=np.float32)
            cls = np.array([])

        # Instances 객체 생성 - segments를 빈 배열로 명시적 초기화
        inst = Instances(
            bboxes=bboxes,
            segments=np.zeros((len(bboxes), 0, 2), dtype=np.float32),  # 빈 segments
            bbox_format="xywh"
        )

        # YOLO가 기대하는 형식으로 샘플 구성
        sample = {
            "img": img_array,
            "cls": cls,
            "instances": inst,
            "ori_shape": img_array.shape[:2],
            "resized_shape": img_array.shape[:2],
            "path": str(img_path),
            "batch_idx": 0
        }

        # LetterBox 적용
        sample = self.letterbox(sample)

        # Tensor로 변환
        img_tensor = torch.from_numpy(sample["img"]).permute(2, 0, 1).contiguous()
        sample["img"] = img_tensor

        return sample

In [37]:
from ultralytics.data import YOLODataset

# 속도 측정용 dataloader

eval_val_gt_labels = load_gt_labels(eval_val_lbl_dir)

fps_test_loader = DataLoader(
    OODDataset(eval_val_img_dir, eval_val_lbl_dir, eval_val_gt_labels),
    batch_size=8,
    shuffle=False,
    collate_fn=custom_collate_fn,
)

Found 792 OOD images


In [ ]:
# 사진 한 장만 포함하는 dataloader

gradio_test_loader_1 = DataLoader(
    YOLODataset(
        # img_path="dataset_prepared_05/eval/val/images/.jpg", #
        data={"names": {0: "어선", 1: "상선", 2: "군함"}},
        imgsz=960,
        augment=False,
        task="detect"
    ),
    batch_size=1,
    shuffle=False
)

In [31]:
def mahalanobis_distance(x, mu, cov_inv):
    """단일 샘플의 마할라노비스 거리 계산"""
    diff = x - mu
    return np.sqrt(np.dot(np.dot(diff, cov_inv), diff.T))

def compute_mahalanobis_scores(model, dataloader, mu, cov_inv, device):
    """테스트 데이터에 대한 마할라노비스 거리를 계산"""
    features_list = []  # 클로저로 사용할 리스트

    def hook_fn(module, input, output):
        pooled = torch.mean(output, dim=(2, 3))
        features_list.append(pooled.detach().cpu())

    handle = neck_layer.register_forward_hook(hook_fn)

    scores = []
    model.eval()

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Computing Mahalanobis scores"):
            imgs = batch["img"].to(device).float() / 255.0
            features_list.clear()

            # 모델 forward
            _ = model(imgs, conf=0.001)

            # Feature가 추출되지 않은 경우 직접 계산
            if not features_list:
                feat_map = model.model.model[:23](imgs)
                pooled = torch.mean(feat_map, dim=(2, 3)).cpu()
                features_list.append(pooled)

            # 배치의 모든 샘플에 대해 거리 계산
            feats = torch.cat(features_list, dim=0).numpy()
            for f in feats:
                d = mahalanobis_distance(f, mu, cov_inv)
                scores.append(d)

    handle.remove()
    return np.array(scores)

In [26]:
def fast_compute_mahalanobis_scores(model, dataloader, mu, cov_inv, device):
    """테스트 데이터에 대한 마할라노비스 거리를 빠르게 계산"""
    # features_list = []  # 클로저로 사용할 리스트

    mu_t = torch.from_numpy(mu).to(device)
    cov_inv_t = torch.from_numpy(cov_inv).to(device)

    # def hook_fn(module, input, output):
    #     pooled = torch.mean(output, dim=(2, 3))
    #     features_list.append(pooled.detach().cpu())

    # handle = neck_layer.register_forward_hook(hook_fn)

    scores = []
    model.eval()

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Fast Mahalanobis scores"):
            imgs = batch["img"].to(device).float() / 255.0
            #  features_list.clear()

            # 모델 forward
            _ = model(imgs, conf=0.001)

            # Head 생략: neck까지만 forward
            feat_map = model.model.model[:23](imgs)

            # GAP: BxC
            pooled = torch.mean(feat_map, dim=(2, 3))

            # Mahalanobis 거리 계산 (GPU)
            diff = pooled - mu_t.unsqueeze(0)
            left = torch.matmul(diff, cov_inv_t)
            dists = torch.sqrt(torch.sum(left * diff, dim=1))

            scores.extend(dists.detach().cpu().numpy())

            # # Feature가 추출되지 않은 경우 직접 계산
            # if not features_list:
            #     feat_map = model.model.model[:23](imgs)
            #     pooled = torch.mean(feat_map, dim=(2, 3)).cpu()
            #     features_list.append(pooled)

            # # 배치의 모든 샘플에 대해 거리 계산
            # feats = torch.cat(features_list, dim=0).numpy()
            # for f in feats:
            #     d = mahalanobis_distance(f, mu, cov_inv)
            #     scores.append(d)

    # handle.remove()
    return np.array(scores)

In [38]:
import time

num_imgs = len(fps_test_loader)

start = time.time()

pred_scores = compute_mahalanobis_scores(model, fps_test_loader, mu, cov_inv, device=0)
pred_labels = (pred_scores > threshold).astype(int)

end = time.time()

inference_time = end - start
print(f"Inference Time: {inference_time} seconds")
print(f"FPS: {num_imgs/inference_time:.4} frame per seconds")

Computing Mahalanobis scores:   0%|          | 0/99 [00:00<?, ?it/s]


0: 544x960 1 어선, 2.0ms
1: 544x960 1 어선, 2.0ms
2: 544x960 1 어선, 2.0ms
3: 544x960 1 어선, 2.0ms
4: 544x960 1 어선, 2.0ms
5: 544x960 1 어선, 2.0ms
6: 544x960 1 어선, 2.0ms
7: 544x960 1 어선, 2.0ms
Speed: 0.0ms preprocess, 2.0ms inference, 0.7ms postprocess per image at shape (1, 3, 544, 960)


Computing Mahalanobis scores:   1%|          | 1/99 [00:00<00:18,  5.23it/s]


0: 544x960 1 어선, 1.9ms
1: 544x960 1 어선, 1.9ms
2: 544x960 2 어선s, 1.9ms
3: 544x960 1 어선, 1.9ms
4: 544x960 1 어선, 1.9ms
5: 544x960 1 어선, 1.9ms
6: 544x960 1 어선, 1.9ms
7: 544x960 1 어선, 1.9ms
Speed: 0.0ms preprocess, 1.9ms inference, 0.6ms postprocess per image at shape (1, 3, 544, 960)


Computing Mahalanobis scores:   2%|▏         | 2/99 [00:00<00:18,  5.25it/s]

...


0: 544x960 1 어선, 1.9ms
1: 544x960 1 어선, 1.9ms
2: 544x960 1 어선, 1.9ms
3: 544x960 1 어선, 1.9ms
4: 544x960 1 어선, 1.9ms
5: 544x960 2 어선s, 1.9ms
6: 544x960 2 어선s, 1.9ms
7: 544x960 2 어선s, 1.9ms
Speed: 0.0ms preprocess, 1.9ms inference, 0.6ms postprocess per image at shape (1, 3, 544, 960)


Computing Mahalanobis scores:  99%|█████████▉| 98/99 [00:49<00:00,  4.89it/s]


0: 544x960 2 어선s, 1.9ms
1: 544x960 4 어선s, 1.9ms
2: 544x960 1 어선, 1.9ms
3: 544x960 1 어선, 1.9ms
4: 544x960 1 어선, 1.9ms
5: 544x960 1 어선, 1.9ms
6: 544x960 1 어선, 1.9ms
7: 544x960 1 어선, 1.9ms
Speed: 0.0ms preprocess, 1.9ms inference, 0.7ms postprocess per image at shape (1, 3, 544, 960)


Computing Mahalanobis scores: 100%|██████████| 99/99 [00:49<00:00,  2.01it/s]

Inference Time: 49.376015186309814 seconds
FPS: 2.005 frame per seconds


In [27]:
import time

num_imgs = len(fps_test_loader)

start = time.time()

pred_scores = fast_compute_mahalanobis_scores(model, fps_test_loader, mu, cov_inv, device=0)
pred_labels = (pred_scores > threshold).astype(int)

end = time.time()

inference_time = end - start
print(f"Inference Time: {inference_time} seconds")
print(f"FPS: {num_imgs/inference_time:.4} frame per seconds")

Fast Mahalanobis scores:   0%|          | 0/25 [00:00<?, ?it/s]


0: 544x960 1 어선, 1.7ms
1: 544x960 1 어선, 1.7ms
2: 544x960 1 어선, 1.7ms
3: 544x960 1 어선, 1.7ms
4: 544x960 1 어선, 1.7ms
5: 544x960 1 어선, 1.7ms
6: 544x960 1 어선, 1.7ms
7: 544x960 1 어선, 1.7ms
8: 544x960 1 어선, 1.7ms
9: 544x960 1 어선, 1.7ms
10: 544x960 2 어선s, 1.7ms
11: 544x960 1 어선, 1.7ms
12: 544x960 1 어선, 1.7ms
13: 544x960 1 어선, 1.7ms
14: 544x960 1 어선, 1.7ms
15: 544x960 1 어선, 1.7ms
16: 544x960 1 어선, 1.7ms
17: 544x960 2 어선s, 1.7ms
18: 544x960 1 어선, 1.7ms
19: 544x960 1 어선, 1.7ms
20: 544x960 1 어선, 1.7ms
21: 544x960 1 어선, 1.7ms
22: 544x960 1 어선, 1.7ms
23: 544x960 1 어선, 1.7ms
24: 544x960 1 어선, 1.7ms
25: 544x960 2 어선s, 1.7ms
26: 544x960 2 어선s, 1.7ms
27: 544x960 1 어선, 1.7ms
28: 544x960 1 어선, 1.7ms
29: 544x960 1 어선, 1.7ms
30: 544x960 2 어선s, 1.7ms
31: 544x960 1 어선, 1.7ms
Speed: 0.0ms preprocess, 1.7ms inference, 1.4ms postprocess per image at shape (1, 3, 544, 960)


Fast Mahalanobis scores:   0%|          | 0/25 [00:00<?, ?it/s]


TypeError: cat() received an invalid combination of arguments - got (Tensor, int), but expected one of:
 * (tuple of Tensors tensors, int dim = 0, *, Tensor out = None)
 * (tuple of Tensors tensors, name dim, *, Tensor out = None)
